In [2]:
import pandas as pd
import math

df = pd.read_csv("Test.csv")

df.head()

,Name
0,PRIYADHARSHINI S
1,PRIYANGA S
2,PRIYANKA R
3,RACHITHA S
4,RAJESH H


In [3]:
num_rows = len(df)
num_qubits = math.ceil(math.log2(num_rows))

print(f"Total rows: {num_rows}")
print(f"Qubits required: {num_qubits}")


Total rows: 66
Qubits required: 7


In [ ]:
df = df.reset_index().rename(columns={"index": "RowIndex"})

df["BinaryIndex"] = df["RowIndex"].apply(
    lambda x: format(x, f"0{num_qubits}b")
)

df.head()


,RowIndex,Name,BinaryIndex
0,0,PRIYADHARSHINI S,0000000
1,1,PRIYANGA S,0000001
2,2,PRIYANKA R,0000010
3,3,RACHITHA S,0000011
4,4,RAJESH H,0000100


In [5]:
mapping = {
    row["BinaryIndex"]: row.drop(["RowIndex", "BinaryIndex"]).to_dict()
    for _, row in df.iterrows()
}

for k, v in list(mapping.items())[:5]:
    print(f"{k} -> {v}")


0000000 -> {'Name': 'PRIYADHARSHINI S'}
0000001 -> {'Name': 'PRIYANGA S'}
0000010 -> {'Name': 'PRIYANKA R'}
0000011 -> {'Name': 'RACHITHA S'}
0000100 -> {'Name': 'RAJESH H'}


In [6]:

search_column = "Name"      
search_value = "SASIKUMAR R"      


target_rows = df[df[search_column] == search_value]

if target_rows.empty:
    print(f"❌ No match found for {search_value} in column {search_column}")
else:
    print(f"✅ Match found:\n{target_rows}")

    # Get RowIndex and BinaryIndex of first match
    target_row = target_rows.iloc[0]
    target_binary = target_row["BinaryIndex"]

    print(f"\nTarget Binary Index: {target_binary}")
    print(f"Target Row Data: {target_row.to_dict()}")


✅ Match found:
    RowIndex         Name BinaryIndex
25        25  SASIKUMAR R     0011001

Target Binary Index: 0011001
Target Row Data: {'RowIndex': 25, 'Name': 'SASIKUMAR R', 'BinaryIndex': '0011001'}
